# Лабораторная работа № 2

# Блочное симметричное шифрование

## Вариант

3(в)

## Задание 

Реализовать систему симметричного блочного шифрования,
позволяющую шифровать и дешифровать файл на диске с использованием
блочного шифра RC6 в режиме шифрования PCBC. 

# Константы

$ w $ - размер слова в битах. Блоки открытого текста и шифротекста имеют длину $ 4w $ - бит (т.к. 4 регистра).

$ r $ - количество раундов. Так же размер расширенного массива ключей $ S $ будет иметь $ t=(2r+4) $ слово. Допустимые значения $ 0\leq r\leq 255 $. 

$ b $ - количество бит в секретном ключе $ K $.

$ K $ - секретный ключ размеров $ b $ байт: $K[0], K[1], ... , K[b-1] $.

In [ ]:
w 